In [11]:
import pandas as pd
import os
import sys

sys.path.append('../src')

import core as c

#### Complete the data

In [12]:
FIRST_SEASON_YEAR = 5
LAST_SEASON_YEAR = 23
league_name = "Premier_league"
code = "E0"

In [13]:
N = 5

In [14]:
selected_columns = [
    'Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HY', 'AY', 'HR', 'AR',
    f'HPTS_avg_{N}', f'APTS_avg_{N}', f'H_gd_{N}', f'A_gd_{N}', f'H_eff_{N}', f'A_eff_{N}', f'HST_avg_{N}', f'AST_avg_{N}', 'target'
]

output_folder = f"../data/processed/"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [19]:
def merge_all_seasons(file_names, output_folder, N):
    merged_df = pd.DataFrame()

    for i in range(len(file_names)):

        # Dodawanie kolumn związanych z bieżącą statystyką
        current_basic_df = pd.read_csv(f"../data/raw/{league_name}/{file_names[i]}.csv")
        current_transformed_df = c.data_transformation(current_basic_df, ["HST", "AST"], window=N)
        current_df = current_transformed_df[selected_columns].copy()  

        # Dodawanie nowych kolumn związanych z ostatnimi sezonami
        if i > 2:
            last_seasons = [pd.read_csv(f"../data/raw/{league_name}/{file_names[j]}.csv") for j in range(i-3, i)]
            teams_status_dict = c.create_teams_status_dict(current_df, last_seasons)
            current_df['H_status'] = current_df['HomeTeam'].apply(lambda team: teams_status_dict.get(team))
            current_df['A_status'] = current_df['AwayTeam'].apply(lambda team: teams_status_dict.get(team))

            current_df = c.calc_h2h_stats(current_df, last_seasons)

        merged_df = pd.concat([merged_df, current_df], ignore_index=True)

    output_file = os.path.join(output_folder, f"{code}_seasons_{FIRST_SEASON_YEAR}_to_{LAST_SEASON_YEAR}.csv")
    merged_df.to_csv(output_file, index=False)
    print(f"Merged data saved to {output_file}")

In [20]:
file_names_to_merge = [f"{code}_{str(year).zfill(2)}_{str(year+1).zfill(2)}" for year in range(FIRST_SEASON_YEAR, LAST_SEASON_YEAR)]
merge_all_seasons(file_names_to_merge, output_folder, N)

Merged data saved to ../data/processed/E0_seasons_5_to_23.csv
